In [1]:
# to add the spiking model codebase to the path
import sys
sys.path.append('..')

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation, writers
# plt.style.use('seaborn-pastel')
plt.rcParams['animation.ffmpeg_path'] ='C:\\ffmpeg\\bin\\ffmpeg.exe'

from brian2 import pA, Hz, second, ms
import numpy as np

from results import Results
from plotting import (MidpointNormalize, weight_matrix, remove_axes_less, prune_figure, prune_figure_less, prune_figure_more,)
from generate import weight_matrix_from_flat_inds_weights
from stats import moving_average, regress_linear, remove_outliers
import matplotlib.font_manager as fm
from analysis import (generate_description, FR_weights, determine_first_convergent_trial, determine_drift_rates,
                      FR_scatter, FR_hist2d, FR_hist1d_compare, FR_weights_std, weights_scatter_matrix,
                      weights_hist1d_compare, weights_matrix_compare, combine_two_sessions, calculate_convergence_index,
                      calculate_net_current_units)

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 10273 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [2]:
targetPath = 'C:/Users/mikejseay/Documents/BrianResults/'
# targetFile = 'classicJercog_1000_0p5_balance_randomUniformSaray_movieTest_2021-05-14-08-22_results'
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_guessZeroActivityWeights2e3p025__2022-01-21-11-04-05_results'

# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_goodCrossHomeoExamp__2022-01-26-15-43-44_results'  # so good*
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_goodCrossHomeoExamp__2022-02-16-14-52-13_results'  # 2T scalar version
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-outer_goodCrossHomeoExamp__2022-02-17-08-22-07_results'  # single term local
targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-outer-homeo_goodCrossHomeoExamp__2022-01-27-07-30-31_results'  # 2T local

# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_goodCrossHomeoExamp__2022-01-26-16-02-50_results'  # 1000 trial version*
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_goodCrossHomeoExamp__2022-01-27-07-30-38_results'

# after clearing everything out
# targetFile = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_resumePrior_seed4explodeDealTestLowAlpha_2022-05-02-12-44-08_results'


R = Results()
R.init_from_file(targetFile, targetPath)

downSampleFactorForE = 0.025
downSampleFactorForI = 0.1

if R.p['useRule'][:11] == 'cross-homeo':
    R.p['alpha2'] = -1e-12
    R.p['tauPlasticityTrials'] = None
if 'wEEScale' not in R.p:
    R.p['wEEScale'] = None
    R.p['wIEScale'] = None
    R.p['wEIScale'] = None
    R.p['wIIScale'] = None

# importantInfoString = 'Name: {}\nEE: {}, IE: {}, EI: {}, II: {}\n tauFR={}, a1={:.4f} pA, a2={:.4f} pA, tauP={}'.format(R.rID,
#     R.p['wEEScale'],R.p['wIEScale'],R.p['wEIScale'],R.p['wIIScale'],
#     R.p['tauUpFRTrials'], R.p['alpha1'] / pA, R.p['alpha2'] / pA, R.p['tauPlasticityTrials'])

wEEInit = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nExc'], R.preEE, R.posEE, R.wEE_init)
wIEInit = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nInh'], R.preIE, R.posIE, R.wIE_init)
wEIInit = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nExc'], R.preEI, R.posEI, R.wEI_init)
wIIInit = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nInh'], R.preII, R.posII, R.wII_init)

wEEFinal = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nExc'], R.preEE, R.posEE, R.wEE_final)
wIEFinal = weight_matrix_from_flat_inds_weights(R.p['nExc'], R.p['nInh'], R.preIE, R.posIE, R.wIE_final)
wEIFinal = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nExc'], R.preEI, R.posEI, R.wEI_final)
wIIFinal = weight_matrix_from_flat_inds_weights(R.p['nInh'], R.p['nInh'], R.preII, R.posII, R.wII_final)

wFullInit = np.block([[wEEInit, wIEInit], [-wEIInit, -wIIInit]])
wFullFinal = np.block([[wEEFinal, wIEFinal], [-wEIFinal, -wIIFinal]])

# get summed presynaptic input to each unit

wEEInitP = np.nansum(wEEInit, 0)
wIEInitP = np.nansum(wIEInit, 0)
wEIInitP = np.nansum(wEIInit, 0)
wIIInitP = np.nansum(wIIInit, 0)

wEEFinalP = np.nansum(wEEFinal, 0)
wIEFinalP = np.nansum(wIEFinal, 0)
wEIFinalP = np.nansum(wEIFinal, 0)
wIIFinalP = np.nansum(wIIFinal, 0)

downSampleNForE = int(downSampleFactorForE * R.p['nExc'])
downSampleNForI = int(downSampleFactorForI * R.p['nInh'])

downSampleE = np.random.choice(R.p['nExc'], size=downSampleNForE, replace=False)
downSampleI = np.random.choice(R.p['nInh'], size=downSampleNForI, replace=False)

In [3]:
R.trialUpFRExcUnits.shape

(3000, 1600)

In [5]:
MSEExcUnits = (R.trialUpFRExcUnits - R.p['setUpFRExc'] / Hz) ** 2
MSEExcUnitsHat = MSEExcUnits.mean(1)
MSEInhUnits = (R.trialUpFRInhUnits - R.p['setUpFRInh'] / Hz) ** 2
MSEInhUnitsHat = MSEInhUnits.mean(1)
MSEUnitsHat = (MSEExcUnitsHat + MSEInhUnitsHat) / 2

In [4]:
print('wEE', np.nanmean(wEEFinal))
print('wIE', np.nanmean(wIEFinal))
print('wEI', np.nanmean(wEIFinal))
print('wII', np.nanmean(wIIFinal))

wEE 174.93898
wIE 145.14908
wEI 209.40926
wII 154.18451


In [5]:
print('totEE: {:.2f} +/- {:.2f}'.format(wEEFinalP.mean(), wEEFinalP.std()))
print('totEI: {:.2f} +/- {:.2f}'.format(wEIFinalP.mean(), wEIFinalP.std()))
print('totIE: {:.2f} +/- {:.2f}'.format(wIEFinalP.mean(), wIEFinalP.std()))
print('totII: {:.2f} +/- {:.2f}'.format(wIIFinalP.mean(), wIIFinalP.std()))

totEE: 69975.60 +/- 2698.33
totEI: 20940.93 +/- 1047.61
totIE: 58059.62 +/- 2131.03
totII: 15418.45 +/- 807.40


In [19]:
netCurrentToE = wEEFinalP - wEIFinalP
netCurrentToI = wIEFinalP - wIIFinalP

In [20]:
netCurrentToE.mean()

34661.465

In [21]:
netCurrentToI.mean()

30106.33

In [22]:
netCurrentToE.std()

3537.8105

In [23]:
netCurrentToI.std()

2711.5173

In [24]:
plt.close('all')

- idea is to draw a movie of the change in average FR and weights across *trials* between trial n1 and n2
- then draw the movie of a single trial (n2)
- then draw the trials from n2 to n3
- then draw the movie of a single trial (n3)

In [3]:
# define things

clipVeryHighFRs = False
yMax = 40
commonXLims = (0, R.trialUpFRExcUnits.shape[0])
commonTrialRange = np.arange(0, R.trialUpFRExcUnits.shape[0])

if clipVeryHighFRs:
    tmptrialUpFRExcUnits = R.trialUpFRExcUnits.copy()
    tmptrialUpFRInhUnits = R.trialUpFRInhUnits.copy()
    tmptrialUpFRExcUnits[R.trialUpFRExcUnits > R.p['maxAllowedFRExc']] = np.nan
    tmptrialUpFRInhUnits[R.trialUpFRInhUnits > R.p['maxAllowedFRInh']] = np.nan
else:
    tmptrialUpFRExcUnits = R.trialUpFRExcUnits.copy()
    tmptrialUpFRInhUnits = R.trialUpFRInhUnits.copy()
    
# for useTrialInd in range(1, len(R.p['saveTrials'])):
useTrialInd = 9
    
useTrialRange = np.arange(0, R.p['saveTrials'][useTrialInd])

trialwEE = R.trialwEE[useTrialRange]
trialwIE = R.trialwIE[useTrialRange]
trialwEI = R.trialwEI[useTrialRange]
trialwII = R.trialwII[useTrialRange]

#     FRExcUnits = tmptrialUpFRExcUnits[useTrialRange, downSampleE]
#     FRInhUnits = tmptrialUpFRInhUnits[useTrialRange, downSampleI]
FRExcUnits = tmptrialUpFRExcUnits[useTrialRange, :][:, downSampleE]
FRInhUnits = tmptrialUpFRInhUnits[useTrialRange, :][:, downSampleI]

trialUpFRExc = R.trialUpFRExc[useTrialRange]
trialUpFRInh = R.trialUpFRInh[useTrialRange]

In [26]:
# FR and weight convergence line graph - static drawing equivalent

In [4]:
fontProp = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')
# fontPropSmall = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')
# fontPropBig = fm.FontProperties(fname='C:\\Users\\mikejseay\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\otf\\MYRIADPRO-REGULAR.otf')

In [5]:
fontProp.set_size(24)
# fontPropSmall.set_size(14)
# fontPropBig.set_size(14)

In [21]:
startTrialInd = 0
endTrialInd = 2000
movAvgWidth = 5
removeFrame = True
removeAxis = False
doSave = True

plt.close('all')
f, ax = plt.subplots(2, 1, num=1, sharex=True, figsize=(6, 10), frameon=False)

if movAvgWidth:
    useFRExc = moving_average(R.trialUpFRExc[startTrialInd:endTrialInd], movAvgWidth)
    useFRInh = moving_average(R.trialUpFRInh[startTrialInd:endTrialInd], movAvgWidth)
else:
    useFRExc = R.trialUpFRExc[startTrialInd:endTrialInd]
    useFRInh = R.trialUpFRInh[startTrialInd:endTrialInd]

ax[0].plot(useFRExc, label='E', color='g', alpha=1, lw=1)
ax[0].plot(useFRInh, label='I', color='r', alpha=1, lw=1)
ax[0].legend(frameon=False, prop=fontProp)
ax[0].hlines(R.p['setUpFRExc'], 0, len(R.trialUpFRExc[startTrialInd:endTrialInd]), ls='--', color='g')
ax[0].hlines(R.p['setUpFRInh'], 0, len(R.trialUpFRInh[startTrialInd:endTrialInd]), ls='--', color='r')
ax[0].set_ylabel('Firing Rate (Hz)', fontproperties=fontProp)
ax[0].set_ylim(0, 25)

ax[1].plot(R.trialwEE[startTrialInd:endTrialInd], label='wEE', color='cyan')
ax[1].plot(R.trialwIE[startTrialInd:endTrialInd], label='wIE', color='purple')
ax[1].plot(R.trialwEI[startTrialInd:endTrialInd], label='wEI', color='cyan', ls='--')
ax[1].plot(R.trialwII[startTrialInd:endTrialInd], label='wII', color='purple', ls='--')
ax[1].legend(frameon=False, prop=fontProp)
ax[1].set_xlabel('Trial #', fontproperties=fontProp)
ax[1].set_ylabel('Weight (pA)', fontproperties=fontProp)
# ax[1].set_ylim(0, ax[1].get_ylim()[1])

for axr in ax:  # christ
    for xtl in axr.get_xticklabels():
        xtl.set_fontproperties(fontProp)
    for ytl in axr.get_yticklabels():
        ytl.set_fontproperties(fontProp)

if removeFrame:
    for axr in ax:
        axr.spines['top'].set_visible(False)
        axr.spines['right'].set_visible(False)

if removeAxis:
    for axr in ax:
        axr.get_xaxis().set_ticks([])
        axr.get_yaxis().set_ticks([])
        axr.axis('off')

f.subplots_adjust(left=0.2, bottom=None, right=None, top=None,)
        
if doSave:
    saveFileName = 'figs/' + R.rID + '_FRweights.pdf'
    f.savefig(saveFileName, transparent=True)

In [30]:
R.trialUpFRExcUnits.shape

(3000, 1600)

In [217]:
targetPath = 'C:/Users/mikejseay/Documents/BrianResults/'
targetFile1 = 'classicJercog_2000_0p25_cross-homeo-pre-scalar_goodCrossHomeoExamp__2022-01-26-15-43-44_results'  # so good
targetFile2 = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_goodCrossHomeoExamp__2022-02-16-14-52-13_results'
targetFile3 = 'classicJercog_2000_0p25_cross-homeo-pre-outer-homeo_goodCrossHomeoExamp__2022-01-27-07-30-31_results'

R1 = Results()
R1.init_from_file(targetFile1, targetPath)
R2 = Results()
R2.init_from_file(targetFile2, targetPath)
R3 = Results()
R3.init_from_file(targetFile3, targetPath)

In [225]:
R1.trialUpFRExcUnits[-1000:, :].mean(0)

array([6.62658969, 3.72086678, 2.00410776, ..., 5.29378884, 2.44211882,
       7.15970701])

In [222]:
import pandas as pd

In [238]:
ExcDf1 = pd.DataFrame({'FR': R1.trialUpFRExcUnits[-1000:, :].mean(0)})
ExcDf2 = pd.DataFrame({'FR': R3.trialUpFRExcUnits[-1000:, :].mean(0)})
ExcDf3 = pd.DataFrame({'FR': R2.trialUpFRExcUnits[-1000:, :].mean(0)})
InhDf1 = pd.DataFrame({'FR': R1.trialUpFRInhUnits[-1000:, :].mean(0)})
InhDf2 = pd.DataFrame({'FR': R2.trialUpFRInhUnits[-1000:, :].mean(0)})
InhDf3 = pd.DataFrame({'FR': R3.trialUpFRInhUnits[-1000:, :].mean(0)})

In [239]:
ExcDf1['label'] = 'E-CH-global'
ExcDf2['label'] = 'E-2T-global'
ExcDf3['label'] = 'E-2T-local'
InhDf1['label'] = 'I-CH-global'
InhDf2['label'] = 'I-2T-global'
InhDf3['label'] = 'I-2T-local'

In [245]:
excDF1 = pd.concat((ExcDf1, ExcDf2))
inhDF1 = pd.concat((InhDf1, InhDf2 ))
excDF2 = pd.concat((ExcDf2, ExcDf3,))
inhDF2 = pd.concat((InhDf2, InhDf3, ))

In [246]:
excDF1.to_csv('varCompareExcDF1.csv')
inhDF1.to_csv('varCompareInhDF1.csv')
excDF2.to_csv('varCompareExcDF2.csv')
inhDF2.to_csv('varCompareInhDF2.csv')

In [218]:
print(R1.trialUpFRExcUnits[-1000:, :].std(1).mean())
print(R1.trialUpFRInhUnits[-1000:, :].std(1).mean())
print(R1.trialUpFRExcUnits[-1000:, :].std(1).mean() / int(R1.p['setUpFRExc']))
print(R1.trialUpFRInhUnits[-1000:, :].std(1).mean() / int(R1.p['setUpFRInh']))

4.136430281245943
12.302581272707295
0.8272860562491886
0.8787558051933783


In [219]:
print(R2.trialUpFRExcUnits[-1000:, :].std(1).mean())
print(R2.trialUpFRInhUnits[-1000:, :].std(1).mean())
print(R2.trialUpFRExcUnits[-1000:, :].std(1).mean() / int(R1.p['setUpFRExc']))
print(R2.trialUpFRInhUnits[-1000:, :].std(1).mean() / int(R1.p['setUpFRInh']))

1.5640721017325272
4.014728417132996
0.3128144203465054
0.28676631550949977


In [220]:
print(R3.trialUpFRExcUnits[-1000:, :].std(1).mean())
print(R3.trialUpFRInhUnits[-1000:, :].std(1).mean())
print(R3.trialUpFRExcUnits[-1000:, :].std(1).mean() / int(R1.p['setUpFRExc']))
print(R3.trialUpFRInhUnits[-1000:, :].std(1).mean() / int(R1.p['setUpFRInh']))

1.6100192869970529
4.175052021548537
0.32200385739941056
0.2982180015391812


# plot with voltage, etc, static equivalent (single trial)

In [13]:
# 0, 9, 15

# plt.close('all')
useTrialInd = 28
doSave = True
removeAxis = True
actualTrialIndex = R.p['saveTrials'][useTrialInd] + 1
frameMult = int(R.p['downSampleVoltageTo'] / R.p['dt'])
useUnitInd = 0
decreaseVoltageBy = 70
FRDT = 0.01  # seconds
voltageYLims = (-90, 10)
targetDisplayedExcUnits = 160
targetDisplayedInhUnits = 40

commonXLims = (0, R.p['duration'] / second)
voltageT = R.selectTrialT[1:]

voltageE = R.selectTrialVExc[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesE = R.selectTrialSpikeExcT[useTrialInd][R.selectTrialSpikeExcI[useTrialInd] == useUnitInd]
vSpikeTimesIndE = (vSpikeTimesE / R.p['dt'] * second / frameMult).astype(int)
voltageE[vSpikeTimesIndE] = 0

voltageI = R.selectTrialVInh[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesI = R.selectTrialSpikeInhT[useTrialInd][R.selectTrialSpikeInhI[useTrialInd] == useUnitInd]
vSpikeTimesIndI = (vSpikeTimesI / R.p['dt'] * second / frameMult).astype(int)
voltageI[vSpikeTimesIndI] = 0

FRT = np.arange(FRDT / 2, R.p['duration'] / second - FRDT / 2, FRDT)
FRE = R.selectTrialFRExc[useTrialInd, :]
FRI = R.selectTrialFRInh[useTrialInd, :]

spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

downSampleE = np.random.choice(R.p['nExc'], size=targetDisplayedExcUnits, replace=False)
downSampleI = np.random.choice(R.p['nInh'], size=targetDisplayedInhUnits, replace=False)
matchingEUnitsBool = np.isin(spikeMonExcI, downSampleE)
matchingIUnitsBool = np.isin(spikeMonInhI, downSampleI)
DownSampleERev = np.full((downSampleE.max() + 1,), np.nan)
DownSampleERev[downSampleE] = np.arange(downSampleE.size)
DownSampleIRev = np.full((downSampleI.max() + 1,), np.nan)
DownSampleIRev[downSampleI] = np.arange(downSampleI.size)
spikesExcT = spikeMonExcT[matchingEUnitsBool]
spikesExcI = DownSampleERev[spikeMonExcI[matchingEUnitsBool].astype(int)]
spikesInhT = spikeMonInhT[matchingIUnitsBool]
spikesInhI = targetDisplayedExcUnits + DownSampleIRev[spikeMonInhI[matchingIUnitsBool].astype(int)]

fig1.clf()
fig1, ax1 = plt.subplots(4, 1, num=2, figsize=(5, 9), gridspec_kw={'height_ratios': [2, 1, 1, 1]}, sharex=True)

srAx, frAx, veAx, viAx = ax1

srAx.set(xlim=commonXLims, ylim=(0, 200), ylabel='Unit Index')      # spike raster
frAx.set(xlim=commonXLims, ylim=(0, 30), ylabel='Firing Rate (Hz)')       # FR
veAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV')  # voltageE
viAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV', xlabel='Time (s)')  # voltageI

if removeAxis:
    for ax in ax1:
        ax.axis('off')
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])

srHandleE, = srAx.plot([], [], c='cyan', ls='', marker='.', markersize=1)
srHandleI, = srAx.plot([], [], c='r', ls='', marker='.', markersize=1)

frHandleE, = frAx.plot([], [], c='cyan', label='Exc', alpha=0.5)
frHandleI, = frAx.plot([], [], c='r', label='Inh', alpha=0.5)

vHandleE, = veAx.plot([], [], c='cyan', lw=.3)
vHandleI, = viAx.plot([], [], c='r', lw=.3)

vHandleE.set_data(voltageT, voltageE)
vHandleI.set_data(voltageT, voltageI)

frHandleE.set_data(FRT, FRE)
frHandleI.set_data(FRT, FRI)
srHandleE.set_data(spikesExcT, spikesExcI)
srHandleI.set_data(spikesInhT, spikesInhI)

fig1.suptitle('Trial {}'.format(actualTrialIndex))

if doSave:
    saveFileName = 'figs/' + R.rID + '_trial{}'.format(actualTrialIndex) + '.pdf'
    fig1.savefig(saveFileName, transparent=True)

In [54]:
from elephant.spike_train_correlation import spike_time_tiling_coefficient
from elephant.spike_train_dissimilarity import van_rossum_distance
from elephant.spike_train_synchrony import spike_contrast
from elephant.statistics import isi, cv

from neo.core import SpikeTrain
import quantities as pq

In [55]:
spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

In [70]:
# traditional CV of ISI analysis... tells you whether the spike train resembles a Poisson process

cvListExc = []
cvListInh = []

for unitInd in range(R.p['nExc']):
    singleUnitBool = spikeMonExcI == unitInd
    singleUnitSpikeTimes = spikeMonExcT[singleUnitBool]
    cvListExc.append(cv(isi(singleUnitSpikeTimes)))
    
for unitInd in range(R.p['nInh']):
    singleUnitBool = spikeMonInhI == unitInd
    singleUnitSpikeTimes = spikeMonInhT[singleUnitBool]
    cvListInh.append(cv(isi(singleUnitSpikeTimes)))

cvList = cvListExc + cvListInh

f, ax = plt.subplots()
ax.hist(cvList, bins=40)
ax.set(xlabel='CV(ISI)', ylabel='# of occurences')

[Text(0.5, 0, 'CV(ISI)'), Text(0, 0.5, '# of occurences')]

In [50]:
# STTC.. it's like a fancy spike timing correlation coefficient

STTC_array = np.full((R.p['nExc'], R.p['nExc']), np.nan)

for excUnit1Ind in range(R.p['nExc']):
    singleUnit1Bool = spikeMonExcI == excUnit1Ind
    singleUnit1SpikeTimes = SpikeTrain(spikeMonExcT[singleUnit1Bool] * s, t_stop=1.5)
    for excUnit2Ind in range(excUnit1Ind + 1):
        singleUnit2Bool = spikeMonExcI == excUnit2Ind
        singleUnit2SpikeTimes = SpikeTrain(spikeMonExcT[singleUnit2Bool] * s, t_stop=1.5)
        STTC_array[excUnit1Ind, excUnit2Ind] = spike_time_tiling_coefficient(singleUnit1SpikeTimes, singleUnit2SpikeTimes)

KeyboardInterrupt: 

In [ ]:
# van rossum distance... 

tau = 10.0 * pq.ms

spikeTrainSeq = []

for unitInd in range(R.p['nExc']):
    spikeTimes 
    spikeTrainSeq.append(spikeMonExcT[spikeMonExcI == unitInd])
    cvListExc.append(cv(isi(singleUnitSpikeTimes)))


In [76]:
# spike-contrast

spikeTrainSeq = []

for unitInd in range(R.p['nExc']):
    spikeTimes = spikeMonExcT[spikeMonExcI == unitInd]
#     spikeTimes = spikeTimes[spikeTimes > 0.25]
    spikeTrain = SpikeTrain(spikeTimes, units='s', t_stop=1.5)
    spikeTrainSeq.append(spikeTrain)
for unitInd in range(R.p['nInh']):
    spikeTimes = spikeMonInhT[spikeMonInhI == unitInd]
#     spikeTimes = spikeTimes[spikeTimes > 0.25]
    spikeTrain = SpikeTrain(spikeTimes, units='s', t_stop=1.5)
    spikeTrainSeq.append(spikeTrain)

In [79]:
synchrony, spike_contrast_trace = spike_contrast(spikeTrainSeq, return_trace=True)

In [80]:
synchrony

0.1729439047190693

#### only other things i can think of:
- a histogram of firing rates / weights
- the weight matrix
- E/I balance for each unit
- up duration over trials

In [102]:
testString1 = '--->-><-><-->-'
testString2 = '>----<'
testString3 = '<<>><'
testString4 = '<<>><>><><>' # remove leading <'s and trailing >'s --> >><>><><

def solution3(s):
    ss = s.replace('-', '').lstrip('<').rstrip('>')
    nl = ss.count('<')
    n = 0
    for c in ss:
        if c == '>':
            n += nl * 2
        else:
            nl -= 1
    return n

In [209]:
# duration

endTrialInd = 2500
movAvgWidth = 9
removeFrame = True
doSave = True

f, ax = plt.subplots(figsize=(6, 4))
ax.plot(moving_average(R.trialUpDur[:endTrialInd], movAvgWidth), color='k')
ax.set_xlabel('Trial #', fontproperties=fontProp)
ax.set_ylabel('Response Duration (s)', fontproperties=fontProp)
for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)

if removeFrame:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_upDur.pdf'
    f.savefig(saveFileName, transparent=True)

In [234]:
plt.close('all')

In [71]:
# histogram of FRs

fontProp.set_size(36)

finalTrials = 1000
nBins = 40
excFRs = R.trialUpFRExcUnits[-finalTrials:, :].mean(0)
inhFRs = R.trialUpFRInhUnits[-finalTrials:, :].mean(0)
removeFrame = True
doSave = True

# minFR = np.fmin(excFRs.min(), inhFRs.min())
# maxFR = np.fmax(excFRs.max(), inhFRs.max())

minFR = 0
maxFR = 30

frBins = np.linspace(minFR, maxFR, nBins)

f, ax = plt.subplots(figsize=(7, 5.5))

ax.hist(R.trialUpFRExcUnits[-1, :], frBins, color='cyan', density=True, histtype='step', label='E')
ax.hist(R.trialUpFRInhUnits[-1, :], frBins, color='r', density=True, histtype='step', label='I')
ax.set_xlabel('FR (Hz)', fontproperties=fontProp)
ax.set_ylabel('prop. of occurences', fontproperties=fontProp)
for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)
ax.legend(frameon=False, prop=fontProp)

if removeFrame:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_FRHist.pdf'
    f.savefig(saveFileName, transparent=True)

In [68]:
# E/I balance to each unit

fontProp.set_size(14)

removeFrame = True
doSave = True

f, axs = plt.subplots(2, 2)

xValSeq = (wEEInitP, wIEInitP, wEEFinalP, wIEFinalP,)
yValSeq = (wEIInitP, wIIInitP, wEIFinalP, wIIFinalP,)

xLblSeq = ('J EE Init (nA)', 'J IE Init (nA)', 'J EE Final (nA)', 'J IE Final (nA)', )
yLblSeq = ('J EI Init (nA)', 'J II Init (nA)', 'J EI Final (nA)', 'J II Final (nA)', )

for axInd, ax in enumerate(axs.ravel()):
    xVals = xValSeq[axInd] / 1e3
    yVals = yValSeq[axInd] / 1e3
    ax.scatter(xVals, yVals, 1)
    pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)
    print(b, r2, np.sqrt(r2), p)
    ax.set_xlabel(xLblSeq[axInd], fontproperties=fontProp)
    ax.set_ylabel(yLblSeq[axInd], fontproperties=fontProp)
    for xtl in ax.get_xticklabels():
        xtl.set_fontproperties(fontProp)
    for ytl in ax.get_yticklabels():
        ytl.set_fontproperties(fontProp)

    if removeFrame:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_unitBalance.pdf'
    f.savefig(saveFileName, transparent=True)

0.057219 0.0007217455017523022 0.026865321545671145 0.2828305859499149
-0.05175126 0.0022392416777353708 0.047320626345552216 0.34517805208497787
0.37886178 0.9522577212102762 0.9758369337190903 0.0
0.36892548 0.9481587865009626 0.9737344537916703 6.81731102614348e-258


In [22]:
# initial and final weight matrices

fontProp.set_size(24)

removeFrame = True
doSave = False
subsetProp = 0.1
nUnitsSub = int(R.p['nUnits'] * subsetProp)
nExcSub = int(R.p['nExc'] * subsetProp)
nInhSub = int(R.p['nInh'] * subsetProp)

if subsetProp:
    wFullInitPlot = np.block([[wEEInit[:nExcSub, :nExcSub], wIEInit[:nExcSub, :nInhSub]],
                          [-wEIInit[:nInhSub, :nExcSub], -wIIInit[:nInhSub, :nInhSub]]])
    wFullFinalPlot = np.block([[wEEFinal[:nExcSub, :nExcSub], wIEFinal[:nExcSub, :nInhSub]],
                           [-wEIFinal[:nInhSub, :nExcSub], -wIIFinal[:nInhSub, :nInhSub]]])
#     wFullInitPlot = np.block([[wEEInit[:nExcSub, :nExcSub], wIEInit[:nExcSub, :nInhSub]],
#                           [wEIInit[:nInhSub, :nExcSub], wIIInit[:nInhSub, :nInhSub]]])
#     wFullFinalPlot = np.block([[wEEFinal[:nExcSub, :nExcSub], wIEFinal[:nExcSub, :nInhSub]],
#                            [wEIFinal[:nInhSub, :nExcSub], wIIFinal[:nInhSub, :nInhSub]]])
else:
    wFullInitPlot = np.block([[wEEInit, wIEInit],
                          [-wEIInit, -wIIInit]])
    wFullFinalPlot = np.block([[wEEFinal, wIEFinal],
                           [-wEIFinal, -wIIFinal]])
#     wFullInitPlot = np.block([[wEEInit, wIEInit],
#                           [wEIInit, wIIInit]])
#     wFullFinalPlot = np.block([[wEEFinal, wIEFinal],
#                            [wEIFinal, wIIFinal]])

allNumbers = np.concatenate((wFullInitPlot, wFullFinalPlot)).ravel()
allNumbersClean = remove_outliers(allNumbers)

vlims = (np.nanmin(allNumbersClean), np.nanmax(allNumbersClean))
# vlims = (0, np.nanmax(allNumbersClean))
    
wFullInitPlot[wFullInitPlot < vlims[0]] = np.nan
wFullInitPlot[wFullInitPlot > vlims[1]] = np.nan

wFullFinalPlot[wFullFinalPlot < vlims[0]] = np.nan
wFullFinalPlot[wFullFinalPlot > vlims[1]] = np.nan

wFullInitPlot[np.isnan(wFullInitPlot)] = 0
wFullFinalPlot[np.isnan(wFullFinalPlot)] = 0

f, ax = plt.subplots(1, 2, figsize=(17, 6))
weight_matrix_local(ax[0], wFullInitPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims, fontproperties=fontProp, removeFrame=removeFrame)
weight_matrix_local(ax[1], wFullFinalPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims, fontproperties=fontProp, removeFrame=removeFrame)

for anax in ax:
    anax.hlines(nExcSub, 0, nUnitsSub, color='white', lw=3)
    anax.vlines(nExcSub, 0, nUnitsSub, color='white', lw=3)
    
if doSave:
    saveFileName = 'figs/' + R.rID + '_weightmats.pdf'
#     saveFileName = 'figs/' + R.rID + '_weightmats.eps'
    f.savefig(saveFileName, transparent=True)

In [14]:
int(wFullInitPlot.shape[0] * R.p['propInh'])

40

In [11]:
wFullInitPlot.shape[0] == wFullInitPlot.shape[1]

True

In [66]:
# 2D weight progression (scatterplot)

removeFrame = True
doSave = True
removeOutliers = False

startTrialInd = 0
endTrialInd = 800

if removeOutliers:
    sd_thresh = 3
    a = R.trialUpFRExc[startTrialInd:endTrialInd]
    b = R.trialUpFRInh[startTrialInd:endTrialInd]
    c = np.sqrt(a ** 2 + b ** 2)
    c_mean = np.nanmean(c)
    c_std = np.nanstd(c)
    c_outlying = np.logical_or(c > c_mean + sd_thresh * c_std, c < c_mean - sd_thresh * c_std)
    xData = a[~c_outlying]
    yData = b[~c_outlying]
else:
    xData = R.trialUpFRExc[startTrialInd:endTrialInd]
    yData = R.trialUpFRInh[startTrialInd:endTrialInd]

zData = np.arange(xData.size) + 2

f, ax = plt.subplots()
s = ax.scatter(xData, yData, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
# ax.plot(R.trialUpFRExc, R.trialUpFRInh, color='k', alpha=0.1)
ax.hlines(R.p['setUpFRInh'] / Hz, ax.get_xlim()[0], ax.get_xlim()[1], linestyles='dotted')
ax.vlines(R.p['setUpFRExc'] / Hz, ax.get_ylim()[0], ax.get_ylim()[1], linestyles='dotted')
ax.set_xlabel('E (Hz)', fontproperties=fontProp)
ax.set_ylabel('I (Hz)', fontproperties=fontProp)

for xtl in ax.get_xticklabels():
    xtl.set_fontproperties(fontProp)
for ytl in ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)

cb = plt.colorbar(s, ax=ax)
cb.ax.set_ylabel('Trial Index', rotation=270, fontproperties=fontProp, labelpad=20)
for ytl in cb.ax.get_yticklabels():
    ytl.set_fontproperties(fontProp)

if doSave:
    saveFileName = 'figs/' + R.rID + '_FRscatter.pdf'
    f.savefig(saveFileName, transparent=True)

In [120]:
test = 0

In [10]:
def weight_matrix_local(ax, values,
                  useCmap='RdBu_r', limsMethod='absmax',
                  xlabel='', ylabel='', clabel='',
                  vlims=None, fontproperties=None, removeFrame=True):
    """ given an axis handle, an array of values, and some optional params,
        visualize a weight matrix in a heat map using imshow
    """
    
    if values.shape[0] == values.shape[1]:
        useAspect = 'equal'
    else:
        useAspect = 'auto'

    i = ax.imshow(values,
                  cmap=getattr(plt.cm, useCmap),
                  aspect=useAspect,
                  interpolation='none')
    ax.set_xlabel(xlabel, fontproperties=fontproperties)
    ax.set_ylabel(ylabel, fontproperties=fontproperties)
    for xtl in ax.get_xticklabels():
        xtl.set_fontproperties(fontproperties)
    for ytl in ax.get_yticklabels():
        ytl.set_fontproperties(fontproperties)
    if removeFrame:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)

    if limsMethod == 'absmax':
        vmax = np.nanmax(np.fabs(values))
        vmin = -vmax
    elif limsMethod == 'minmax':
        vmax, vmin = np.nanmax(values), np.nanmin(values)
    elif limsMethod == 'custom':
        vmin, vmax = vlims

    if vmin != -vmax and vmin < 0 and vmax > 0:
        norm = MidpointNormalize(vmin, vmax, 0)
    else:
        norm = False

    i.set_clim(vmin, vmax)
    if norm:
        i.set_norm(norm)

    cb = plt.colorbar(i, ax=ax)
    cb.ax.set_ylabel(clabel, rotation=270, fontproperties=fontproperties, labelpad=20)
    for ytl in cb.ax.get_yticklabels():
        ytl.set_fontproperties(fontProp)
    cb.outline.set_visible((not removeFrame))

In [9]:
# plot with voltage, etc, video equivalent (single trial)

frameMult = int(R.p['downSampleVoltageTo'] / R.p['dt'])
useUnitInd = 0
decreaseVoltageBy = 70
FRDT = 0.01  # seconds
voltageYLims = (-90, 10)
targetDisplayedExcUnits = 100
targetDisplayedInhUnits = 100

commonXLims = (0, R.p['duration'] / second)
voltageT = R.selectTrialT[1:]

voltageE = R.selectTrialVExc[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesE = R.selectTrialSpikeExcT[useTrialInd][R.selectTrialSpikeExcI[useTrialInd] == useUnitInd]
vSpikeTimesIndE = (vSpikeTimesE / R.p['dt'] * second / frameMult).astype(int)
voltageE[vSpikeTimesIndE] = 0

voltageI = R.selectTrialVInh[useTrialInd, :] - decreaseVoltageBy
vSpikeTimesI = R.selectTrialSpikeInhT[useTrialInd][R.selectTrialSpikeInhI[useTrialInd] == useUnitInd]
vSpikeTimesIndI = (vSpikeTimesI / R.p['dt'] * second / frameMult).astype(int)
voltageI[vSpikeTimesIndI] = 0

FRT = np.arange(FRDT / 2, R.p['duration'] / second - FRDT / 2, FRDT)
FRE = R.selectTrialFRExc[useTrialInd, :]
FRI = R.selectTrialFRInh[useTrialInd, :]

spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

downSampleE = np.random.choice(R.p['nExc'], size=targetDisplayedExcUnits, replace=False)
downSampleI = np.random.choice(R.p['nInh'], size=targetDisplayedInhUnits, replace=False)
matchingEUnitsBool = np.isin(spikeMonExcI, downSampleE)
matchingIUnitsBool = np.isin(spikeMonInhI, downSampleI)
DownSampleERev = np.full((downSampleE.max() + 1,), np.nan)
DownSampleERev[downSampleE] = np.arange(downSampleE.size)
DownSampleIRev = np.full((downSampleI.max() + 1,), np.nan)
DownSampleIRev[downSampleI] = np.arange(downSampleI.size)
spikesExcT = spikeMonExcT[matchingEUnitsBool]
spikesExcI = DownSampleERev[spikeMonExcI[matchingEUnitsBool].astype(int)]
spikesInhT = spikeMonInhT[matchingIUnitsBool]
spikesInhI = targetDisplayedExcUnits + DownSampleIRev[spikeMonInhI[matchingIUnitsBool].astype(int)]

fig1, ax1 = plt.subplots(4, 1, num=1, figsize=(5, 9), gridspec_kw={'height_ratios': [2, 1, 1, 1]}, sharex=True)

srAx, frAx, veAx, viAx = ax1

srAx.set(xlim=commonXLims, ylim=(0, 200), ylabel='Unit Index')      # spike raster
frAx.set(xlim=commonXLims, ylim=(0, 30), ylabel='Firing Rate (Hz)')       # FR
veAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV')  # voltageE
viAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV', xlabel='Time (s)')  # voltageI

srHandleE, = srAx.plot([], [], c='g', ls='', marker='.', markersize=1)
srHandleI, = srAx.plot([], [], c='r', ls='', marker='.', markersize=1)

frHandleE, = frAx.plot([], [], c='g', label='Exc', alpha=0.5)
frHandleI, = frAx.plot([], [], c='r', label='Inh', alpha=0.5)

vHandleE, = veAx.plot([], [], c='g', lw=.3)
vHandleI, = viAx.plot([], [], c='r', lw=.3)

def init():
    srHandleE.set_data([], [])
    srHandleI.set_data([], [])
    frHandleE.set_data([], [])
    frHandleI.set_data([], [])
    vHandleE.set_data([], [])
    vHandleI.set_data([], [])

    print('initalized animation')

    return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

def animate(i):
    vHandleE.set_data(voltageT[:i], voltageE[:i])
    vHandleI.set_data(voltageT[:i], voltageI[:i])

    fri = int(i / 10)
    frHandleE.set_data(FRT[:fri], FRE[:fri])
    frHandleI.set_data(FRT[:fri], FRI[:fri])

    currentTime = voltageT[i]
    sBoolE = spikesExcT <= currentTime
    sBoolI = spikesInhT <= currentTime
    srHandleE.set_data(spikesExcT[sBoolE], spikesExcI[sBoolE])
    srHandleI.set_data(spikesInhT[sBoolI], spikesInhI[sBoolI])

    print('animated frame', i)

    return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

useFPS = 30
totalFrames = int(np.round((voltageT[-1] * second) * useFPS * Hz))
useInterval = int(np.round(voltageT.size / totalFrames))
framesToSave = np.arange(0, voltageT.size, useInterval)

anim = FuncAnimation(fig1, animate, init_func=init,
                     frames=framesToSave, blit=True, repeat=False)
# anim.save('trial' + str(useTrialInd) + '.gif')

Writer = writers['ffmpeg']
writer = Writer(fps=useFPS)
anim.save('trial' + str(R.p['saveTrials'][useTrialInd] + 1) + '.mp4', writer=writer)
fig1.clear()

initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
ani

animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
an

TypeError: 'NoneType' object is not subscriptable

initalized animation
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485


In [9]:
# plot with voltage, etc, video equivalent (multiple trials)

frameMult = int(R.p['downSampleVoltageTo'] / R.p['dt'])
useUnitInd = 0
decreaseVoltageBy = 70
FRDT = 0.01  # seconds
voltageYLims = (-90, 10)
targetDisplayedExcUnits = 100
targetDisplayedInhUnits = 100

commonXLims = (0, R.p['duration'] / second)
voltageT = R.selectTrialT[1:]

for useTrialInd in range(len(R.p['saveTrials'])):

    voltageE = R.selectTrialVExc[useTrialInd, :] - decreaseVoltageBy
    vSpikeTimesE = R.selectTrialSpikeExcT[useTrialInd][R.selectTrialSpikeExcI[useTrialInd] == useUnitInd]
    vSpikeTimesIndE = (vSpikeTimesE / R.p['dt'] * second / frameMult).astype(int)
    voltageE[vSpikeTimesIndE] = 0

    voltageI = R.selectTrialVInh[useTrialInd, :] - decreaseVoltageBy
    vSpikeTimesI = R.selectTrialSpikeInhT[useTrialInd][R.selectTrialSpikeInhI[useTrialInd] == useUnitInd]
    vSpikeTimesIndI = (vSpikeTimesI / R.p['dt'] * second / frameMult).astype(int)
    voltageI[vSpikeTimesIndI] = 0

    FRT = np.arange(FRDT / 2, R.p['duration'] / second - FRDT / 2, FRDT)
    FRE = R.selectTrialFRExc[useTrialInd, :]
    FRI = R.selectTrialFRInh[useTrialInd, :]

    spikeMonExcT = R.selectTrialSpikeExcT[useTrialInd]
    spikeMonExcI = R.selectTrialSpikeExcI[useTrialInd]
    spikeMonInhT = R.selectTrialSpikeInhT[useTrialInd]
    spikeMonInhI = R.selectTrialSpikeInhI[useTrialInd]

    downSampleE = np.random.choice(R.p['nExc'], size=targetDisplayedExcUnits, replace=False)
    downSampleI = np.random.choice(R.p['nInh'], size=targetDisplayedInhUnits, replace=False)
    matchingEUnitsBool = np.isin(spikeMonExcI, downSampleE)
    matchingIUnitsBool = np.isin(spikeMonInhI, downSampleI)
    DownSampleERev = np.full((downSampleE.max() + 1,), np.nan)
    DownSampleERev[downSampleE] = np.arange(downSampleE.size)
    DownSampleIRev = np.full((downSampleI.max() + 1,), np.nan)
    DownSampleIRev[downSampleI] = np.arange(downSampleI.size)
    spikesExcT = spikeMonExcT[matchingEUnitsBool]
    spikesExcI = DownSampleERev[spikeMonExcI[matchingEUnitsBool].astype(int)]
    spikesInhT = spikeMonInhT[matchingIUnitsBool]
    spikesInhI = targetDisplayedExcUnits + DownSampleIRev[spikeMonInhI[matchingIUnitsBool].astype(int)]

    fig1, ax1 = plt.subplots(4, 1, num=1, figsize=(5, 9), gridspec_kw={'height_ratios': [2, 1, 1, 1]}, sharex=True)

    srAx, frAx, veAx, viAx = ax1

    srAx.set(xlim=commonXLims, ylim=(0, 200), ylabel='Unit Index')      # spike raster
    frAx.set(xlim=commonXLims, ylim=(0, 30), ylabel='Firing Rate (Hz)')       # FR
    veAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV')  # voltageE
    viAx.set(xlim=commonXLims, ylim=voltageYLims, ylabel='mV', xlabel='Time (s)')  # voltageI

    srHandleE, = srAx.plot([], [], c='g', ls='', marker='.', markersize=1)
    srHandleI, = srAx.plot([], [], c='r', ls='', marker='.', markersize=1)

    frHandleE, = frAx.plot([], [], c='g', label='Exc', alpha=0.5)
    frHandleI, = frAx.plot([], [], c='r', label='Inh', alpha=0.5)

    vHandleE, = veAx.plot([], [], c='g', lw=.3)
    vHandleI, = viAx.plot([], [], c='r', lw=.3)

    def init():
        srHandleE.set_data([], [])
        srHandleI.set_data([], [])
        frHandleE.set_data([], [])
        frHandleI.set_data([], [])
        vHandleE.set_data([], [])
        vHandleI.set_data([], [])

        print('initalized animation')

        return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

    def animate(i):
        vHandleE.set_data(voltageT[:i], voltageE[:i])
        vHandleI.set_data(voltageT[:i], voltageI[:i])

        fri = int(i / 10)
        frHandleE.set_data(FRT[:fri], FRE[:fri])
        frHandleI.set_data(FRT[:fri], FRI[:fri])

        currentTime = voltageT[i]
        sBoolE = spikesExcT <= currentTime
        sBoolI = spikesInhT <= currentTime
        srHandleE.set_data(spikesExcT[sBoolE], spikesExcI[sBoolE])
        srHandleI.set_data(spikesInhT[sBoolI], spikesInhI[sBoolI])

        print('animated frame', i)

        return srHandleE, srHandleI, frHandleE, frHandleI, vHandleE, vHandleI, 

    useFPS = 30
    totalFrames = int(np.round((voltageT[-1] * second) * useFPS * Hz))
    useInterval = int(np.round(voltageT.size / totalFrames))
    framesToSave = np.arange(0, voltageT.size, useInterval)

    anim = FuncAnimation(fig1, animate, init_func=init,
                         frames=framesToSave, blit=True, repeat=False)
    # anim.save('trial' + str(useTrialInd) + '.gif')

    Writer = writers['ffmpeg']
    writer = Writer(fps=useFPS)
    anim.save('trial' + str(R.p['saveTrials'][useTrialInd] + 1) + '.mp4', writer=writer)
    fig1.clear()

initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
ani

animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
an

TypeError: 'NoneType' object is not subscriptable

initalized animation
initalized animation
animated frame 0
animated frame 33
animated frame 66
animated frame 99
animated frame 132
animated frame 165
animated frame 198
animated frame 231
animated frame 264
animated frame 297
animated frame 330
animated frame 363
animated frame 396
animated frame 429
animated frame 462
animated frame 495
animated frame 528
animated frame 561
animated frame 594
animated frame 627
animated frame 660
animated frame 693
animated frame 726
animated frame 759
animated frame 792
animated frame 825
animated frame 858
animated frame 891
animated frame 924
animated frame 957
animated frame 990
animated frame 1023
animated frame 1056
animated frame 1089
animated frame 1122
animated frame 1155
animated frame 1188
animated frame 1221
animated frame 1254
animated frame 1287
animated frame 1320
animated frame 1353
animated frame 1386
animated frame 1419
animated frame 1452
animated frame 1485
